In [32]:
import pysam
import os
import pickle
from colections import Counter
import pandas as pd
import re
import matplotlib.pyplot as plt

In [ ]:
#searcHPV workdir
workdir = 'MCV_fusion'

In [33]:
mcvDic = {'VP2': [[465, 1190]],
 'VP1': [[1156, 2427]],
 'large_T': [[2503, 4722], [5154, 5387]],
 'small_T': [[4827, 5387]]}

In [34]:
mcvList = []
for gene in mcvDic:
    for each in mcvDic[gene]:
        mcvList.append([gene,each[0],each[1]])

In [35]:
mcvDf = pd.DataFrame(mcvList, columns = ['gene','start','end'])

In [37]:
mcvDf

,gene,start,end
0,VP2,465,1190
1,VP1,1156,2427
2,large_T,2503,4722
3,large_T,5154,5387
4,small_T,4827,5387


In [38]:
#MCV gene directions
direction = ['+','+','-','-','-']
mcvDf['direction'] = ['+','+','-','-','-']

In [40]:
i = 0
newStart = []
newEnd = []
for each in direction:
    
    if each == '-':
        newStart.append(mcvList[i][2])
        newEnd.append(mcvList[i][1])
    else:
        newStart.append(mcvList[i][1])
        newEnd.append(mcvList[i][2])
    i += 1

In [42]:
mcvDf['newStart'] = newStart
mcvDf['newEnd'] = newEnd

In [43]:
mcvDf = mcvDf.drop(['start','end'],axis = 1)

In [44]:
mcvDf.to_csv('mcvDf.csv')

In [46]:
siteDic = {}
sampleList = ['83108','83092']
sampleName = ['HNSpore4055T','HNSpore4003T']
i = 0
for sample in sampleList:
    path = f'{workdir}/Sample_{sample}/call_fusion_virus'
    res = f'{path}/filteredSelectedContig.pickle'
    with open(res,'rb') as res:
        resDic = pickle.load(res)
        if resDic != {}:
            siteDic[sampleName[i]] = resDic
            
    i += 1


In [47]:
geneDic = {}
gtfFileGenome = 'Homo_sapiens.GRCh38.105.transcript.gtf'
with open(gtfFileGenome) as genome:
    rows = genome.read().rstrip().split('\n')
    for eachRow in rows:
        hasGeneName = False
        for each in eachRow.split('\t')[8].split(';'):
            if each.split('"')[0] == ' gene_name ':
                geneName = each.split('"')[1]
                hasGeneName = True
            if each.split('"')[0] == ' transcript_id ':
                trID = each.split('"')[1]
        if not hasGeneName:
            geneName = trID
        chro = eachRow.split('\t')[0]
        start = eachRow.split('\t')[3]
        end = eachRow.split('\t')[4]
        strand = eachRow.split('\t')[6]
        if trID not in geneDic:
            geneDic[trID] = [geneName,chro,start,end,strand]
        else:
            print(eachRow)

In [50]:
genePrimaryList = []
geneSiteList = []
count = 0
for sample in siteDic:
    for ins in siteDic[sample]:
        chro = ins.split('.')[0]
        site = ins.split('.')[1]
        geneNameDic = {}
        flag = False
        for gene in geneDic:
            if geneDic[gene][1] == chro and int(geneDic[gene][2]) <= int(site) and int(geneDic[gene][3]) > int(site):
                start = geneDic[gene][2]
                end = geneDic[gene][3]
                dirct = geneDic[gene][4]
                geneNameDic[geneDic[gene][0]] = [start,end,dirct]
                flag = True
        genePrimaryList += geneNameDic.keys()
        for gene in geneNameDic:
            chro = ins.split('.')[0]
            site = ins.split('.')[1]
            geneSiteList.append([sample,gene,chro,site] + geneNameDic[gene])
        if flag == True:
            count += 1
    

In [51]:
countDic = dict(Counter(genePrimaryList))

In [52]:
genePlotDfList = []
for each in geneSiteList:
    genePlotDfList.append(each)
genePlotDf = pd.DataFrame(genePlotDfList,columns= ['sample','gene','chr','ins','start','end','direction'])
genePlotDf.to_csv('geneTrack_MCV.csv')

In [56]:
script = 'extractContig.sh'
with open(script,'w') as scriptfile:
    for each in genePlotDfList:
        chro = each[2]
        ins = each[3]
        sample = each[0]
        gene = each[1]
        path = work_dir #working dir of searcHPV
        contigfile = f'{path}/call_fusion_virus/ContigsSequence.fa'
        resfile = f'{path}/{gene}.fa'
        scriptfile.write(f'grep -A2 {chro}.{ins} {contigfile} >> {resfile};\n')


In [57]:
script = 'alignContig.sh'
with open(script,'w') as scriptfile:
    
    scriptfile.write('''module load Bioinformatics;
module load bwa;
module load samtools;
''')
    for each in genePlotDfList:
        gene = each[1]
        sample = each[0]
        path = 'workdir'
        resfile = f'{path}/{gene}.fa'
        ref = 'hg_mcv.fa'
        scriptfile.write(f'''bwa mem -R '@RG\\tID:hpv\\tSM:mcv\\tLB:mcv\\tPL:ILLUMINA' -M -t 8 {ref} {resfile} > {path}/{gene}.sam;
    samtools view -bhS {path}/{gene}.sam > {path}/{gene}.bam;
    samtools sort -@ 8 -o {path}/{gene}.sort.bam {path}/{gene}.bam;
    samtools index {path}/{gene}.sort.bam;''')


In [58]:
#exon dictionary
file = "=../data/Homo_sapiens.GRCh38.105.exon.gtf"
exonDic = {}
with open(file) as inputFile:
    inputFile.readline()
    rows = inputFile.read().split('\n')
    for row in rows:
        if row != '':
            hasExonNum = False
            hasGeneName = False
            for each in row.split('\t')[8].split(';'):
                if each.split('"')[0] == ' exon_number ':
                    exonNum = each.split('"')[1]
                    hasExonNum = True
                if each.split('"')[0] == ' gene_name ':
                    geneName = each.split('"')[1]
                    hasGeneName = True
                if each.split('"')[0] == ' transcript_id ':
                    trID = each.split('"')[1]
                if each.split('"')[0] == ' exon_id ':
                    exonID = each.split('"')[1]
            if not hasGeneName:
                geneName = trID
            if hasExonNum:
                exonStart = row.split('\t')[3]
                exonEnd = row.split('\t')[4]
                for each in genePlotDfList:
                    if each[1] == geneName:
                        exonDic[exonID] = [geneName,exonStart,exonEnd,exonNum,trID]



In [ ]:
#open Sample_2655-CB-90 HEMGN contigs alignment
annotation = []
inserts_annotation = []
for eachline in genePlotDfList:
    sample = eachline[0]
    gene = eachline[1]
    # if sample == 'Sample_2655-CB-89' and gene == 'KLF13':
    dirct = eachline[6]
    ins = eachline[3]
    path = 'workdir'
    bam = f'{path}/{gene}.sort.bam'
    chro = eachline[2]
    genestart = eachline[4]
    geneend = eachline[5]
    genedir = eachline[6]
    samfile = pysam.AlignmentFile(bam,'rb')
    inserts = []

    for read in samfile.fetch(chro,int(genestart),int(geneend)):
        start  = read.pos
        length = read.query_alignment_length
        fullLength = read.infer_read_length()
        name = read.qname
        contig = name.split('.')[2]
        contigins = name.split('.')[1]
        #print(sample,chro,ins)
        for eachContig in siteDic[sample][chro + '.' + contigins]:
            if eachContig[0] == contig:
                if eachContig[-1] != 'lowConfidence':
                    mcvSite = eachContig[-1]
                else:
                    mcvSite = eachContig[-2]

        mcvGeneBreak = ''
        for eachMCVgene in mcvDic:
            for eachSlot in mcvDic[eachMCVgene]:
                if int(mcvSite) >= int(eachSlot[0]) and int(mcvSite) <= int(eachSlot[1]):
                    mcvGeneBreak = mcvGeneBreak + eachMCVgene + ';'
        if read.is_reverse:
            strand = '-'
        else:
            strand = '+'
        #add flag
        flag = 'intron'
        exonNumList = ''
        for exon in exonDic:
            exonStart = int(exonDic[exon][1])
            exonEnd = int(exonDic[exon][2])
            exonNum = int(exonDic[exon][3])
            
            if start <=  int(exonEnd) and start + length >= exonStart:
                flag = 'exon'
                exonNumList = exonNumList + exon + ';'
                
        
               
                    
        #resolve contig hum components
        cigar = read.cigar
        infoList = [sample,gene,name,contig,'hum',flag,genedir,fullLength,mcvSite,'',mcvGeneBreak,contigins,genestart,geneend,length,exonNumList,strand]
    
        
        if cigar[0][0] == 0:
            #matched part, if strand of contig is different from gene direction, reverse compliment the cigar, switch the start and end
            if infoList[6] == '+':
                pointStart = 0
                pointEnd = cigar[0][1]
            else:
                pointStart = fullLength
                pointEnd = fullLength - cigar[0][1]
                
        else:
            pointStart = cigar[1][1]
            if cigar[-1][0] == 4 or cigar[-1][0] == 5:
                pointEnd = fullLength - cigar[-1][1]
            else:
                pointEnd = fullLength
            if infoList[6] == '-':
                pointStart = fullLength - cigar[0][1]
                pointEnd = fullLength - pointEnd
        inserts.append([pointStart,pointEnd] + infoList)
        
        
        if read.has_tag('SA'):
            saTag = read.get_tag('SA')
        else:
            print(read)
        #print(saTag)
        aligns = saTag.split(';')
        for align in aligns:
            if align != '':
                supchro = align.split(',')[0]
                if supchro == 'NC_010277.2':
                    suppos = align.split(',')[1]
                    supstrand = align.split(',')[2]
                    supcigar = align.split(',')[3]
                    slots = re.split(r'[A-Z]',supcigar)[:-1]
                    slotCigars = re.split(r'\d+',supcigar)[1:]
                    #print(align,slots,slotCigars)
                    i = 0
                    #add flag
                    if slotCigars[0] == 'M':
                        match = int(slots[0])
                        supEnd = mcvSite - match
                    else:
                        match = int(slots[1])
                        supEnd = mcvSite + match
                    newFlag = []
                    hasEach1 = False
                    for each in mcvDic:
                        for eachone in mcvDic[each]:
                            if int(suppos) >= int(eachone[0]) and int(suppos) <= int(eachone[1]):
                                each1 = each
                                newFlag.append(each1)
                                hasEach1 = True
                            if int(suppos) + match >= int(eachone[0]) and int(suppos) + match <= int(eachone[1]):
                                each2 = each
                                if each2 != each1 and hasEach1:
                                    newFlag.append(each2)
                                elif not hasEach1:
                                    newFlag.append(each2)

                    if newFlag == []:
                        newFlag = ''
                    else:
                        newFlag = ';'.join(newFlag)
                    #print(newFlag)
                    infoList = [sample,gene,name,contig,'mcpv',newFlag,genedir,fullLength,mcvSite,supEnd,mcvGeneBreak,contigins,genestart,geneend,length,'',supstrand]
                    if slotCigars[0] == 'M':
                        if supstrand == strand: #read converted to consistent cigar to original human alignment
                            if infoList[6] == '+': #read on hum refence was not converted
                                pointStart = 0
                                pointEnd = int(slots[0])
                            else: #read on hum refence was not converted
                                pointStart = fullLength
                                pointEnd = fullLength - int(slots[0])
                        else: #read have inverted cigar to original human alignment
                            if infoList[6] == '+': #read on hum refence was not converted, converted to human alignments
                                pointStart = fullLength
                                pointEnd = fullLength - int(slots[0])
                            else:#read on hum refence was not converted, don't converted
                                pointStart = 0
                                pointEnd = int(slots[0])
                                
                        inserts.append([pointStart,pointEnd] + infoList)
                        #print(supcigar,supstrand,strand)
                        #print([pointStart,pointEnd] + infoList)
                    else:

                        if supstrand == strand: #read converted to consistent cigar to original human alignment
                            pointStart = int(slots[0])
                            if slotCigars[-1] == 'S' or slotCigars[-1] == 'H':
                                pointEnd = fullLength - int(slots[-1])
                            else:
                                pointEnd = fullLength
                            if infoList[6] == '-': #read on hum refence was converted
                                pointStart = fullLength - pointStart
                                pointEnd = fullLength - pointEnd
                               
                        else:

                            if slotCigars[-1] == 'S' or slotCigars[-1] == 'H':
                                pointStart = int(slots[-1])
                            else:
                                pointStart = 0
                            pointEnd = fullLength - int(slots[0])
                            #base on direction change order of start and end
                            #print(infoList[6])
                            #print(pointStart)
                            if infoList[6] == '-': #read on hum refence was not converted, converted to human alignments
                                pointStart = fullLength - pointStart
                                pointEnd = fullLength - pointEnd
                            inserts.append([pointEnd,pointStart] + infoList)
                            
        

        annotation.append([sample,gene,sample,contig,chro,start,length,strand,contigins])
        #print(inserts)
    #print(suppos,int(suppos) + match,inserts)
    inserts_annotation += inserts
    #pd.DataFrame(inserts,columns = ['name','contig','start','end','type','feature','geneDirection','fullLength']).to_csv(outputFile2)

In [61]:
outputFile1 = 'annotation.csv'
outputFile2 = 'inserts.csv'

In [62]:
annotation_df = pd.DataFrame(annotation,columns = ['sampleID','gene','sampleName','contig','chr','start','length','strand','ins'])
annotation_df.to_csv(outputFile1)

In [63]:
inserts_df = pd.DataFrame(inserts_annotation,columns = ['start','end','sample','gene','name','contig','type','feature','geneDirection','fullLength','hpvSite','hpvEnd','hpvGene','ins','geneStart','geneEnd','length','exonNum','strand'] )
inserts_df.to_csv(outputFile2)